## Importing the DataFrames

In [3]:
import pickle
import pandas as pd

In [4]:
all_data = pd.read_pickle("../data/processed/all_data.pkl")
sp_map = pd.read_pickle("../data/processed/sp_map.pkl")
with open("../data/processed/week_dfs.pkl", "rb") as f:
   week_dfs = pickle.load(f)
with open("../data/processed/week_sp_dfs.pkl", "rb") as f:
   week_sp_dfs = pickle.load(f)

## Creating Maps

In [ ]:
from datetime import datetime

def get_color_relative_to_week(time, start, end):
    # Normalize the time to a value between 0 and 1
    relative_position = (time - start) / (end - start)

    # Convert to a color (you can adjust the logic to change color mapping)
    hue = int(255 * relative_position)  # Map to a color spectrum
    color = f'hsl({hue}, 100%, 50%)'  # Using HSL color model
    return color

In [6]:
import folium

def creating_maps(df, started_tracked, sp=''):
    week = df['week_number'].iloc[0]
    
    # Assuming df is your DataFrame and 'tracked_at' is converted to datetime
    start_week = df[started_tracked].min()
    end_week = df[started_tracked].max()
    
    # Calculate the average latitude and longitude for the initial map center
    avg_latitude = df['latitude'].mean()
    avg_longitude = df['longitude'].mean()
    
    # Create a Folium map centered around the average coordinates
    map_folium = folium.Map(location=[avg_latitude, avg_longitude], zoom_start=12)
    
    for _, row in df.iterrows():
        time = row[started_tracked].to_pydatetime()
        color = get_color_relative_to_week(time, start_week, end_week)
    
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=3,
            color=color,
            fill=True,
            fill_color=color,
            popup=row[started_tracked]
        ).add_to(map_folium)
    
    map_file_path = '../maps/maps_per_week'+sp+'/map'+sp+''+ str(week) +'th_week_of_the_year.html'
    map_folium.save(map_file_path)
    return None

In [ ]:
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore', message='Discarding nonzero nanoseconds in conversion')

for i in tqdm(range(len(week_dfs))):  
    creating_map(week_dfs[i],'tracked_at')

In [ ]:
warnings.filterwarnings('ignore', message='Discarding nonzero nanoseconds in conversion')
for i in tqdm(range(len(week_sp_dfs))):  
    creating_map(week_sp_dfs[i],'started_at',sp='_SP')

In [ ]:
# Calculate the average latitude and longitude for the initial map center
avg_latitude = all_data['latitude'].mean()
avg_longitude = all_data['longitude'].mean()

# Create a Folium map centered around the average coordinates
map_folium = folium.Map(location=[avg_latitude, avg_longitude], zoom_start=11)

# Adding the data points to the map
for _, row in all_data.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color='red',
        fill=True,
        fill_color='red'
    ).add_to(map_folium)

# Saving the map to an HTML file
map_file_path = '../maps/maps_folium_all_data/ALL_POINTS.html'
map_folium.save(map_file_path)

In [ ]:
# Calculate the average latitude and longitude for the initial map center
avg_latitude = sp_map['latitude'].mean()
avg_longitude = sp_map['longitude'].mean()

# Create a Folium map centered around the average coordinates
map_folium = folium.Map(location=[avg_latitude, avg_longitude], zoom_start=12)

# Adding the data points to the map
for _, row in sp_map.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color='red',
        fill=True,
        fill_color='red'
    ).add_to(map_folium)

# Saving the map to an HTML file
map_file_path = '../maps/maps_folium_all_data/SP_points.html'
map_folium.save(map_file_path)